In [1]:
import pandas as pd
import numpy as np
import warnings


warnings.filterwarnings('ignore', category=Warning)

In [2]:
##Comes from BOM File
df=pd.read_csv(r'Trimming Generation input.csv')

In [3]:
column_mapping = {
    'Sort Order': 'MY_LEVEL',
    'FactCode': 'COMP_ORG',
    'Primary Item Code': 'ASSEMB_ITEM_CODE',
    'Original Description': 'ASSEMB_ITEM_DESC',
    'Original UOM': 'ASSEMB_UOM',
    'Ingredient': 'COMP_ITEM_CODE',
    'Description.1': 'COMP_DESC',
    'final_output': 'COMP_QTY',
    'UOM.1': 'COMP_UOM',
    'Item Code': 'SUB_ASSEMBLY_ITEM_CODE',
    'Description': 'SUB_ASSEMBLY_ITEM_DESC',
    'UOM': 'SUB_ASSEMBLY_UOM',
    'Packing flag': 'Packing flag'
}
df = df.rename(columns=column_mapping)

In [4]:
df_t=df[df['COMP_DESC'].str.contains('CHICKEN CALIBRATED FILLET')]

df_t[['Lower_Level', 'Upper_Level']] = df_t['COMP_DESC'].str.extract(r'YIELD:\s*(\d+)\s*-\s*(\d+)')

df_t['Lower_Level'] = pd.to_numeric(df_t['Lower_Level'], errors='coerce')
df_t['Upper_Level'] = pd.to_numeric(df_t['Upper_Level'], errors='coerce')

# Calculate the average
df_t['AVG'] = (df_t['Lower_Level'] + df_t['Upper_Level']) / 200

df_t['Total_Chicken']=df_t['COMP_QTY']/df_t['AVG']

df_t['Trimming(in KG)']=df_t['Total_Chicken']-df_t['COMP_QTY']
df_t.head(2)

,SUB_ASSEMBLY_ITEM_CODE,SUB_ASSEMBLY_ITEM_DESC,Qty,SUB_ASSEMBLY_UOM,Line Type,COMP_ITEM_CODE,COMP_DESC,COMP_UOM,Qty.1,Unit Cost,...,uniq_comb,Qty.1/Qty,multiplier,intermediate_output,COMP_QTY,Lower_Level,Upper_Level,AVG,Total_Chicken,Trimming(in KG)
4138,3001370261,CHICKEN FILLET BURGER,700.030,KGM,Ingredient,1001100022,CHICKEN CALIBRATED FILLET - YIELD: 31-40 %,KGM,549.50,13.940121,...,4001370200_NFC,0.784966,11.19,8.783774,8.783774,31,40,0.355,24.743024,15.959251
4706,3001237165,STRIPS JORDAN NEW,318.412,KGM,Ingredient,1001100024,CHICKEN CALIBRATED FILLET - YIELD: 51-60 %,KGM,170.32,0.000000,...,4001370244_GFC,0.534904,11.75,6.285127,6.285127,51,60,0.555,11.324554,5.039427


In [5]:
df_t.to_csv('29-04-24_Trimming_Output.csv',index=False)

In [6]:
# Group the data by ASSEMB_ITEM_CODE and count unique COMP_ORG
grouped = df_t.groupby('ASSEMB_ITEM_CODE')['COMP_ORG'].nunique().reset_index()

# Filter out the ASSEMB_ITEM_CODE that appear in more than one COMP_ORG
filtered = grouped[grouped['COMP_ORG'] > 1]

# Filter out the ASSEMB_ITEM_CODE that appear in only one COMP_ORG
single_comp_org = grouped[grouped['COMP_ORG'] == 1]['ASSEMB_ITEM_CODE'].tolist()

# Initialize an empty list to store the results
results_list = []

# Iterate through the filtered DataFrame and collect the results
for index, row in filtered.iterrows():
    assem_item_code = row['ASSEMB_ITEM_CODE']
    comp_org_count = row['COMP_ORG']
    
    # Get the COMP_ORG_IDs for the ASSEMB_ITEM_CODE and convert them to strings
    comp_org_ids = df_t[df_t['ASSEMB_ITEM_CODE'] == assem_item_code]['COMP_ORG'].unique()
    comp_org_ids_str = [str(x) for x in comp_org_ids]
    
    # Create a dictionary for the current ASSEMB_ITEM_CODE and its COMP_ORG_IDs
    result_dict = {
        'ASSEMB_ITEM_CODE': assem_item_code,
        'COMP_ORG_COUNT': comp_org_count,
        'COMP_ORG_IDS': ', '.join(comp_org_ids_str)
    }
    
    # Append the dictionary to the results list
    results_list.append(result_dict)

# Add the single COMP_ORG ASSEMB_ITEM_CODE to the results_list
for assem_item_code in single_comp_org:
    comp_org_count = 1
    
    # Get the COMP_ORG_ID for the ASSEMB_ITEM_CODE and convert it to a string
    comp_org_id = df_t[df_t['ASSEMB_ITEM_CODE'] == assem_item_code]['COMP_ORG'].unique()[0]
    comp_org_ids_str = str(comp_org_id)
    
    # Create a dictionary for the current ASSEMB_ITEM_CODE and its COMP_ORG_ID
    result_dict = {
        'ASSEMB_ITEM_CODE': assem_item_code,
        'COMP_ORG_COUNT': comp_org_count,
        'COMP_ORG_IDS': comp_org_ids_str
    }
    
    # Append the dictionary to the results list
    results_list.append(result_dict)

# Convert the results list to a new DataFrame
df_results = pd.DataFrame(results_list)

In [7]:
df_results.to_csv('trimming_in_1 or more factory.csv',index=False)